In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
import random
from collections import OrderedDict 
from tqdm import trange
%matplotlib inline

# Network Robustness Analysis

Questions: 
- How many nodes can we remove while the network preserves its functioning condition?
- How many nodes do we need to remove to fragment the network into isolated components?

*Notes: 
- Attack order was computed beforehand instead of on the fly, because it drastically increased computation time.
- Should we take avg. or max. values?*

In [ ]:
G = nx.read_gml('Graphs/airlines.gml')
# 1% of data (rounded, 14 nodes will be left at the end)
REMOVAL_SIZE = int(len(G)/100)
# upper bound % of node to be removed
REMOVAL_COUNT = 100

In [ ]:
def percentage_of_removed_links(G):
    return int(len(G)/100)

def random_attack(g, removal_size, node_removal_algorithm, img_name):
    
    graph = g.copy()
    # network metrics
    degree = []
    #avg_out_degree = []
    SCC = []
    avg_closeness = []
    avg_betweenness = []
    avg_clustering = []
    avg_eigvec_centrality = []
    num_of_steps = []
    
    for percentage in trange(REMOVAL_COUNT):
        # compute avg. degree
        degree.append(sum([d for (n, d) in nx.degree(graph)]) / float(graph.number_of_nodes()))
        #avg_out_degree.append(sum([d for (n, d) in graph.out_degree()]) / float(graph.number_of_nodes()))
        # clustering
        avg_clustering.append(nx.average_clustering(graph))
        # largest strongly connected components
        SCC.append(len(max(nx.strongly_connected_components(graph), key=len)))
        # closeness
        avg_closeness.append(np.average(list(nx.closeness_centrality(graph).values())))
        # betweenness
        avg_betweenness.append(np.average(list(nx.betweenness_centrality(graph).values())))
        # eigenvector centrality (DOES NOT CONVERGE!)
        #avg_eigvec_centrality.append(np.average(list(nx.eigenvector_centrality(graph).values())))
        # remove node according to given node removal algorithm
        # remove node at random
        for i in range(removal_size):
            target = random.choice(list(graph.nodes.keys()))
            graph.remove_node(target)
        # keep track of steps
        num_of_steps.append(percentage)
    
    # plot statistics
    fig, axs = plt.subplots(3, figsize=(15,15))
    fig.suptitle('Network statistics after attack')
    axs[0].plot(num_of_steps, degree, color='blue', label='avg. degree')
    #axs[0].plot(num_of_steps, avg_out_degree, color='cyan', label='avg. out degree')
    axs[0].legend()
    axs[0].set_ylabel('k')
    axs[0].set_xlabel('% of removed nodes')
    axs[0].set_yscale('log')
    axs[1].plot(num_of_steps, SCC, color='red', label='SCC')
    axs[1].set_ylabel('Giant component size')
    axs[1].set_xlabel('% of removed nodes')
    axs[1].set_yscale('log')
    axs[1].legend()
    axs[2].plot(num_of_steps, avg_closeness, color='green', label='avg. closeness')
    axs[2].plot(num_of_steps, avg_betweenness, color='orange', label='avg. betweenness')
    axs[2].plot(num_of_steps, avg_clustering, color='purple', label='avg. clustering')
    #axs[2].plot(num_of_steps, avg_eigvec_centrality, color='brown', label='avg. eigvec. centr.')
    axs[2].set_ylabel('CC,CB,C,EC')
    axs[2].set_xlabel('% of removed nodes')
    axs[2].set_yscale('log')
    axs[2].legend()
    fig.savefig('Figures/robustness/' + img_name)
    fig.show()

## Random Attacks

In [ ]:
def remove_random_node(g, removal_size):
    # remove node at random
    for i in range(removal_size):
        target = random.choice(list(g.nodes.keys()))
        g.remove_node(target)

In [ ]:
# load network
G = nx.read_gml('Graphs/airlines.gml')
# random attack
random_attack(G, REMOVAL_SIZE, remove_random_node, 'random.pdf')

## Targeted Attack

In [ ]:
def percentage_of_removed_links(G):
    return int(len(G)/100)

def get_by_index(sorted_dict, i):
   return sorted_dict[i][0]
 

def targeted_attack(g, sorted_dict, removal_size, img_name):
    
    graph = g.copy()
    node_idx = 0
    # network metrics
    degree = []
    #avg_out_degree = []
    SCC = []
    avg_closeness = []
    avg_betweenness = []
    avg_clustering = []
    avg_eigvec_centrality = []
    num_of_steps = []
    
    for percentage in trange(REMOVAL_COUNT):
        # compute avg. degree
        degree.append(sum([d for (n, d) in nx.degree(graph)]) / float(graph.number_of_nodes()))
        #avg_out_degree.append(sum([d for (n, d) in graph.out_degree()]) / float(graph.number_of_nodes()))
        # clustering
        avg_clustering.append(nx.average_clustering(graph))
        # largest strongly connected components
        SCC.append(len(max(nx.strongly_connected_components(graph), key=len)))
        # closeness
        avg_closeness.append(np.average(list(nx.closeness_centrality(graph).values())))
        # betweenness
        avg_betweenness.append(np.average(list(nx.betweenness_centrality(graph).values())))
        # eigenvector centrality (DOES NOT CONVERGE!)
        # avg_eigvec_centrality.append(np.average(list(nx.eigenvector_centrality(graph,max_iter=100).values())))
        # keep track of steps
        num_of_steps.append(percentage)
        # remove nodes
        for i in range(REMOVAL_SIZE):
            target = get_by_index(sorted_dict, node_idx)
            graph.remove_node(target)
            node_idx +=1
    
    # plot statistics
    fig, axs = plt.subplots(3, figsize=(15,15))
    fig.suptitle('Network statistics after attack')
    axs[0].plot(num_of_steps, degree, color='blue', label='avg. degree')
    #axs[0].plot(num_of_steps, avg_out_degree, color='cyan', label='avg. out degree')
    axs[0].legend()
    axs[0].set_ylabel('k')
    axs[0].set_xlabel('% of removed nodes')
    axs[0].set_yscale('log')
    axs[1].plot(num_of_steps, SCC, color='red', label='SCC')
    axs[1].set_ylabel('Giant component size')
    axs[1].set_xlabel('% of removed nodes')
    axs[1].set_yscale('log')
    axs[1].legend()
    axs[2].plot(num_of_steps, avg_closeness, color='green', label='avg. closeness')
    axs[2].plot(num_of_steps, avg_betweenness, color='orange', label='avg. betweenness')
    axs[2].plot(num_of_steps, avg_clustering, color='purple', label='avg. clustering')
    #axs[2].plot(num_of_steps, avg_eigvec_centrality, color='brown', label='avg. eigvec. centr.')
    axs[2].set_ylabel('CC,CB,C') #,EC
    axs[2].set_xlabel('% of removed nodes')
    axs[2].set_yscale('log')
    axs[2].legend()
    fig.savefig('Figures/robustness/' + img_name)
    fig.show()

### Betweenness Centrality

In [ ]:
# load network
G = nx.read_gml('Graphs/airlines.gml')
# Optimisation: precompute closeness dict instead of doing it continuously on the fly
sorted_betweenness = sorted(dict(nx.betweenness_centrality(G)).items(), key=itemgetter(1), reverse=True)
# betweenness attack
targeted_attack(G, sorted_betweenness, REMOVAL_SIZE, 'betweenness.pdf')

### Closeness Centrality

In [ ]:
# load network
G = nx.read_gml('Graphs/airlines.gml')
# Optimisation: precompute closeness dict instead of doing it continuously on the fly
sorted_closeness = sorted(dict(nx.closeness_centrality(G)).items(), key=itemgetter(1), reverse=True)
# closeness attack
targeted_attack(G, sorted_closeness, REMOVAL_SIZE, 'closeness.pdf')

### Degree

In [ ]:
# load network
G = nx.read_gml('Graphs/airlines.gml')
# Optimisation: precompute degree dict instead of doing it continuously on the fly
sorted_degree_dict = sorted(dict(nx.degree(G)).items(), key=itemgetter(1), reverse=True)
# degree attack
targeted_attack(G, sorted_degree_dict, REMOVAL_SIZE, 'degree.pdf')

### Eigenvector Centrality

*Note: Does not converge!*

In [ ]:
# load network
G = nx.read_gml('Graphs/airlines.gml')
# Optimisation: precompute degree dict instead of doing it continuously on the fly
sorted_eigvec_dict = sorted(dict(nx.eigenvector_centrality(G)).items(), key=itemgetter(1), reverse=True)
# eigenvector centrality attack
targeted_attack(G, sorted_eigvec_dict, REMOVAL_SIZE, 'eigvec.pdf')